### Mathematical model

We have a finite number of oscillators with natural frequency $\omega_i$ and a phase angle $\phi_i$.

The coupling the evolution (ODE) is now given by

\begin{equation}
     \dot \phi_i = \omega_i + \sum_{j\neq i} k_{ij} \sin(\phi_i-\phi_j-\alpha) + \eta_i
\end{equation}

where the $k_{ij}$ are the coupling constants, $\alpha$ is the phase delay and $\eta_i$ is standard white noise.

We take here $\alpha=0$, $k_{ij}=k$, $\omega_i=\omega$ and no noise: 

\begin{equation}
     \dot \phi_i = \omega +  k \sum_{j\neq i}\sin(\phi_i-\phi_j)
\end{equation}

### Visualizations

(1) By considering $e^{i\phi_i}$ we can visualise the evolution on the unit circle.

(2) The interaction is given through the so-called *order parameter*

\begin{equation}
    r = \sum_i e^{i\phi_i} 
\end{equation}

which is also used to measure the degree of synchronisation. This is is just the mean of all the oscillators.

### Numerical solutions

**[Runge-Kutta]**

Following https://laszukdawid.com/2017/03/02/kuramoto-in-python/, we use scipy integration ODE, with 'dopri5' (explicit runge-kutta method of order (4)5.

https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.ode.html

Hairer, Norsett, Wanner_Solving Ordinary Differential Equations.


**[explicit Euler]**

From https://hdietert.github.io/static/kuramoto-animation/kuramoto.html, a simple forward Euler scheme (explicit) should be enough to numerical solve the ODE. If

\begin{eqnarray}
        \dot y(t) = f(t, y(t)) \quad \text{and} \quad y(t_0) = y_0 \\
\end{eqnarray}

then  by setting $t_{n+1} = h + t_n $ we have

\begin{eqnarray}
        y_{n+1} = y_{n} + h f(t_n, y_n).
\end{eqnarray}




#### set visualizations

In [ ]:
import matplotlib.pyplot as plt

def viz_kuramoto(phase, dt):

    dot_phase = np.diff(phase)/dt
    r = np.exp(1j*phase).sum(axis=0)
    
    fig, axes = plt.subplots(nrows=2*len(W)+2, ncols=1, figsize=(18, 18))

    for comp in range(len(W)):
        axes[comp].plot(T, phase[comp],'b-')
        axes[comp].set_title('$\phi_%i(t)$'%(comp+1))
        axes[comp].set_ylim(phase.min(), phase.max())

    for comp in range(len(W)):
        axes[comp+len(W)].plot(T[:-1], dot_phase[comp],'r-')
        axes[comp+len(W)].set_title('$\dot\phi_%i(t)$'%(comp+1))
        axes[comp+len(W)].set_ylim(dot_phase.min(), dot_phase.max())

    axes[2*len(W)].set_title('Order parameter r')
    axes[2*len(W)].scatter(r.real, r.imag, c='y')
    axes[2*len(W)].set_aspect('equal', 'box')

    axes[2*len(W)+1].plot(T, r.real,'g-', label='Re[r_(t)]')
    axes[2*len(W)+1].plot(T, r.imag,'c-', label='Im[r_(t)]')
    axes[2*len(W)+1].set_title('Order parameter r')

    plt.legend()
    plt.tight_layout()
    plt.show()

#### define initial values and parameters

In [ ]:
import numpy as np

dtype = np.float32

t0, t1, dt = 0, 20, 0.1
T = np.arange(t0, t1, dt)

seed = 666
rnd = np.random.RandomState(seed)

init_phase = 2*np.pi*rnd.random_sample(size=5)
W = 1*np.ones(len(init_phase)) 
K = 10/len(init_phase) 

print(init_phase)

## [Runge-Kutta]

#### define problem to integrate (right hand size of ODE)

In [ ]:
def kuramoto_ODE(t, y, arg):
    """General Kuramoto ODE of m'th harmonic order.
       Argument `arg` = (w, k), with
        w -- iterable frequency
        k -- 3D coupling matrix, unless 1st order
        """
    w, k = arg
    yt = y[:, None]
    dy = y - yt
    phase = w.astype(dtype)
    # add noise
    # phase += 0.0001*np.random.uniform(len(phase))
    phase += np.sum(k*np.sin(dy), axis=1)
    return phase

#### define scipy integrator

In [ ]:
from scipy.integrate import ode

# Set ODE integrator (explicit runge-kutta method of order (4)5)
kODE = ode(kuramoto_ODE)
kODE.set_integrator("dopri5")

# Set parameters into model
kODE.set_initial_value(init_phase, T[0])
kODE.set_f_params((W, K))



#### do the integration

In [ ]:
# Run ODE integrator
y = np.empty((len(W), len(T)))
for idx, _t in enumerate(T[1:]):
    y[:,idx] = kODE.y
    kODE.integrate(_t)
y[:,-1] = kODE.y

viz_kuramoto(y, dt)

## [explicit Euler]

In [ ]:
def f(y):
    yt = y[:, None]
    dy = y - yt
    out = W.astype(dtype)
    # add noise
    # phase += 0.0001*np.random.uniform(len(phase))
    out += np.sum(K*np.sin(dy), axis=1)
    return out


y = np.empty((len(W), len(T)))
y[:, 0] = init_phase
for idx in range(1, len(T)):
    y[:, idx] = y[:,idx-1] + dt*f(y[:,idx-1])

viz_kuramoto(y, dt)